In [8]:
import numpy as np
import pandas as pd
from tensorflow.keras import layers, models, callbacks
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
import seaborn as sns
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

def extract_advanced_features(data):
    data['acceleration'] = data['speed'].diff().fillna(0)
    data['turning_angle'] = data['direction'].diff().fillna(0)
    return data
    
def build_refined_model(input_shape):
    model = models.Sequential()
    
    # CNN layers
    model.add(layers.Conv1D(32, 3, activation='relu', input_shape=input_shape, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Conv1D(64, 3, activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2))
    
    # GRU layer
    model.add(layers.GRU(64, return_sequences=False))
    
    # fully connected layers
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1, activation='sigmoid'))  # Binary classification
    
    # compile model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def train_model(X, y, epochs=20, batch_size=32, test_size=0.2, random_state=42):
    """
    trains the CNN-GRU model and evaluates performance
    """
    # split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    # build the model
    model = build_refined_model((X_train.shape[1], X_train.shape[2]))
    
    # callbacks for early stopping
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    # train the model
    history = model.fit(X_train, y_train, 
                        epochs=epochs, 
                        batch_size=batch_size, 
                        validation_data=(X_test, y_test), 
                        callbacks=[early_stopping],
                        verbose=1)
    
    # predict probabilities and classify based on threshold 0.5
    y_pred_proba = model.predict(X_test)
    y_pred = (y_pred_proba > 0.5).astype("int32")
    
    # calculate metrics
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall:", recall_score(y_test, y_pred))
    print("F1-Score:", f1_score(y_test, y_pred))
    print("ROC-AUC:", roc_auc_score(y_test, y_pred_proba))
    
    # confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plot_confusion_matrix(cm)
    
    return model, history, X_train, X_test, y_test

# plot Confusion Matrix
def plot_confusion_matrix(cm):
    """
    plots the confusion matrix using seaborn heatmap.
    """
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['True 0', 'True 1'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('CNN-GRU Confusion Matrix')
    plt.show()

# plot ROC Curve
def plot_roc_curve(y_test, y_pred_proba):
    """
    plots the ROC curve.
    """
    from sklearn.metrics import roc_curve, auc
    
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc='lower right')
    plt.show()


# Main
if __name__ == "__main__":
    # Load dataset
    dataset_csv = "fly_features_extended.csv"  # Path to your feature CSV file
    dataset = pd.read_csv(dataset_csv)
    
    # Feature Engineering
    dataset = extract_advanced_features(dataset)
    
    # Normalize features
    feature_columns = ["dx", "dy", "speed", "direction", "acceleration", "turning_angle"]
    scaler = StandardScaler()
    dataset[feature_columns] = scaler.fit_transform(dataset[feature_columns])
    
    # Prepare features (X) and labels (y)
    X = dataset[feature_columns].values
    y = dataset["label"].values

    # Handle missing values
    imputer = SimpleImputer(strategy="mean")
    X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
    
    # Reshape X for CNN-GRU (samples, time_steps, features)
    X = X.reshape(X.shape[0], 1, X.shape[1])
    
    # Train the model
    model, history, X_train, X_test, y_test = train_model(X, y)
    
    # Plot ROC curve
    y_pred_proba = model.predict(X_test)  # Get predicted probabilities
    plot_roc_curve(y_test, y_pred_proba)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'